# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

## Load the Data

Start by importing the data from the pickle file.

In [4]:
# TODO: Implement load the data here.
work_dir = 'C:\\Users\\Ali\\Desktop\\Car_ND-Lab2\\'
with open(work_dir+'train.p', 'rb') as f:
    data = pickle.load(f)
    
X_all = data['features']
y_all = data['labels']
print(X_all.shape, y_all.shape)

(39209, 32, 32, 3) (39209,)


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [5]:
# TODO: Use `train_test_split` here.
X_train, X_val, y_train, y_val = train_test_split(X_all,y_all,test_size=0.15, random_state=832289,stratify=y_all)


In [6]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [7]:
# TODO: Implement data normalization here.

print(X_train.max(), X_train.min())
print(X_val.max(), X_val.min())

X_train_norm = X_train - 127.5
X_train_norm /= 255
X_val_norm = X_val - 127.5
X_val_norm /= 255

print(X_train_norm.max(), X_train_norm.min())
print(X_val_norm.max(), X_val_norm.min())

X_train = X_train_norm
X_val = X_val_norm


255 0
255 0
0.5 -0.5
0.5 -0.5


In [8]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [9]:
# TODO: Build a two-layer feedforward neural network with Keras here.

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout

X_train = X_train.reshape(-1,3072)
X_val = X_val.reshape(-1,3072)

model = Sequential()
model.add(Dense(128,input_dim=3072))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


Using TensorFlow backend.


In [10]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [11]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_1 (None, 3072) (None, 128) <function linear at 0x00000250456AFA60>
activation_1 (None, 128) (None, 128) <function relu at 0x00000250456AF840>
dense_2 (None, 128) (None, 43) <function linear at 0x00000250456AFA60>
activation_2 (None, 43) (None, 43) <function softmax at 0x00000250456AF598>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [13]:
# TODO: Compile and train the model here.

from sklearn.preprocessing import OneHotEncoder

# defining the OneHotEncoder class, reshaping the label data and fitting/transforming to ohe format
enc = OneHotEncoder(dtype=np.float32)
y_train_ohe = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_val_ohe = enc.fit_transform(y_val.reshape(-1,1)).toarray()

#
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

b_size = 128
n_epochs = 5
history = model.fit(X_train, y_train_ohe, 
                    batch_size=b_size, 
                    nb_epoch=n_epochs, 
                    verbose=1, 
                    validation_data=(X_val, y_val_ohe))


Train on 33327 samples, validate on 5882 samples
Epoch 1/5
33327/33327 [==============================] - 4s - loss: 0.3445 - acc: 0.9124 - val_loss: 0.3786 - val_acc: 0.8885
Epoch 2/5
33327/33327 [==============================] - 4s - loss: 0.3052 - acc: 0.9195 - val_loss: 0.3636 - val_acc: 0.8977
Epoch 3/5
33327/33327 [==============================] - 4s - loss: 0.2901 - acc: 0.9226 - val_loss: 0.3065 - val_acc: 0.9226
Epoch 4/5
33327/33327 [==============================] - 4s - loss: 0.2403 - acc: 0.9384 - val_loss: 0.3430 - val_acc: 0.8934
Epoch 5/5
33327/33327 [==============================] - 4s - loss: 0.2328 - acc: 0.9372 - val_loss: 0.2934 - val_acc: 0.9160


In [14]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (0.9160)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [15]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

with open(work_dir+'train.p', 'rb') as f:
    data = pickle.load(f)
    
X_all = data['features']
y_all = data['labels']

X_train, X_val, y_train, y_val = train_test_split(X_all,y_all,test_size=0.15, random_state=832289,stratify=y_all)

X_train_norm = X_train - 127.5
X_train_norm /= 255
X_val_norm = X_val - 127.5
X_val_norm /= 255

X_train = X_train_norm
X_val = X_val_norm

# defining the OneHotEncoder class, reshaping the label data and fitting/transforming to ohe format
enc = OneHotEncoder(dtype=np.float32)
y_train_ohe = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_val_ohe = enc.fit_transform(y_val.reshape(-1,1)).toarray()


print(X_train.shape, y_train.shape, y_train_ohe.shape, X_val.shape, y_val.shape, y_val_ohe.shape)

img_rows = 32
img_cols = 32
nb_filters = 32
kernel_size = (3, 3)

# TODO: Compile and train the model here.

from keras.layers import Convolution2D, Flatten

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                       border_mode='valid',
                       input_shape=(img_rows,img_cols,3)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

b_size = 128
n_epochs = 5
history = model.fit(X_train, y_train_ohe, 
                    batch_size=b_size, 
                    nb_epoch=n_epochs, 
                    verbose=1, 
                    validation_data=(X_val, y_val_ohe))

(33327, 32, 32, 3) (33327,) (33327, 43) (5882, 32, 32, 3) (5882,) (5882, 43)
Train on 33327 samples, validate on 5882 samples
Epoch 1/5
33327/33327 [==============================] - 51s - loss: 0.8621 - acc: 0.7816 - val_loss: 0.3270 - val_acc: 0.9189
Epoch 2/5
33327/33327 [==============================] - 52s - loss: 0.2043 - acc: 0.9493 - val_loss: 0.2197 - val_acc: 0.9459
Epoch 3/5
33327/33327 [==============================] - 52s - loss: 0.1152 - acc: 0.9716 - val_loss: 0.1995 - val_acc: 0.9487
Epoch 4/5
33327/33327 [==============================] - 52s - loss: 0.0845 - acc: 0.9796 - val_loss: 0.1864 - val_acc: 0.9548
Epoch 5/5
33327/33327 [==============================] - 53s - loss: 0.0592 - acc: 0.9848 - val_loss: 0.1710 - val_acc: 0.9628


In [16]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (0.9628)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [17]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

from keras.layers import MaxPooling2D

pool_size = (2,2)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                       border_mode='valid',
                       input_shape=(img_rows,img_cols,3)))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


# TODO: Compile and train the model here.

model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

b_size = 128
n_epochs = 5
history = model.fit(X_train, y_train_ohe, 
                    batch_size=b_size, 
                    nb_epoch=n_epochs, 
                    verbose=1, 
                    validation_data=(X_val, y_val_ohe))

Train on 33327 samples, validate on 5882 samples
Epoch 1/5
33327/33327 [==============================] - 27s - loss: 1.2013 - acc: 0.6930 - val_loss: 0.4138 - val_acc: 0.9021
Epoch 2/5
33327/33327 [==============================] - 27s - loss: 0.2706 - acc: 0.9386 - val_loss: 0.2323 - val_acc: 0.9432
Epoch 3/5
33327/33327 [==============================] - 27s - loss: 0.1431 - acc: 0.9695 - val_loss: 0.1719 - val_acc: 0.9597
Epoch 4/5
33327/33327 [==============================] - 27s - loss: 0.0935 - acc: 0.9803 - val_loss: 0.1544 - val_acc: 0.9653
Epoch 5/5
33327/33327 [==============================] - 26s - loss: 0.0647 - acc: 0.9870 - val_loss: 0.1439 - val_acc: 0.9636


In [18]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (0.9636)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [19]:
# TODO: Re-construct the network and add dropout after the pooling layer.

from keras.layers import Dropout

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                       border_mode='valid',
                       input_shape=(img_rows,img_cols,3)))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


# TODO: Compile and train the model here.

model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

b_size = 128
n_epochs = 5
history = model.fit(X_train, y_train_ohe, 
                    batch_size=b_size, 
                    nb_epoch=n_epochs, 
                    verbose=1, 
                    validation_data=(X_val, y_val_ohe))

Train on 33327 samples, validate on 5882 samples
Epoch 1/5
33327/33327 [==============================] - 33s - loss: 1.3845 - acc: 0.6357 - val_loss: 0.5027 - val_acc: 0.8796
Epoch 2/5
33327/33327 [==============================] - 34s - loss: 0.3760 - acc: 0.9057 - val_loss: 0.2619 - val_acc: 0.9446
Epoch 3/5
33327/33327 [==============================] - 34s - loss: 0.2235 - acc: 0.9442 - val_loss: 0.1979 - val_acc: 0.9531
Epoch 4/5
33327/33327 [==============================] - 34s - loss: 0.1647 - acc: 0.9586 - val_loss: 0.1602 - val_acc: 0.9633
Epoch 5/5
33327/33327 [==============================] - 33s - loss: 0.1305 - acc: 0.9680 - val_loss: 0.1396 - val_acc: 0.9696


In [20]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (0.9696)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [28]:
from keras.layers import AveragePooling2D

img_rows = 32
img_cols = 32

model = Sequential()

nb_filters1 = 32
k_size1 = (5, 5)
model.add(Convolution2D(nb_filters1, k_size1[0], k_size1[1],
                       border_mode='valid',
                       input_shape=(img_rows,img_cols,3)))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Activation('relu'))

nb_filters2 = 64
k_size2 = (3, 3)
model.add(Convolution2D(nb_filters2, k_size2[0], k_size2[1],
                       border_mode='valid',
                       input_shape=(28,28,32)))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Dense(43))
model.add(Activation('softmax'))


# compiling and training the model

model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

b_size = 128
n_epochs = 20
history = model.fit(X_train, y_train_ohe, 
                    batch_size=b_size, 
                    nb_epoch=n_epochs, 
                    verbose=1, 
                    validation_data=(X_val, y_val_ohe))

Train on 33327 samples, validate on 5882 samples
Epoch 1/20
33327/33327 [==============================] - 40s - loss: 2.3056 - acc: 0.3951 - val_loss: 1.0602 - val_acc: 0.7450
Epoch 2/20
33327/33327 [==============================] - 37s - loss: 0.7603 - acc: 0.8227 - val_loss: 0.4154 - val_acc: 0.9208
Epoch 3/20
33327/33327 [==============================] - 39s - loss: 0.3993 - acc: 0.9144 - val_loss: 0.2262 - val_acc: 0.9577
Epoch 4/20
33327/33327 [==============================] - 46s - loss: 0.2560 - acc: 0.9463 - val_loss: 0.1504 - val_acc: 0.9726
Epoch 5/20
33327/33327 [==============================] - 53s - loss: 0.1784 - acc: 0.9636 - val_loss: 0.1002 - val_acc: 0.9815
Epoch 6/20
33327/33327 [==============================] - 53s - loss: 0.1336 - acc: 0.9726 - val_loss: 0.0783 - val_acc: 0.9854
Epoch 7/20
33327/33327 [==============================] - 53s - loss: 0.1030 - acc: 0.9795 - val_loss: 0.0671 - val_acc: 0.9867
Epoch 8/20
33327/33327 [==============================]

**Best Validation Accuracy:** 

Changed kernel size to 5x5 (0.9725)

Added a 2nd convolution layer with 3x3 kernel + max pooling before + relu before (0.9830)

increased the number of filters in the 2nd conv layer to 64 (0.9849)

added a dropout after the first convolution's relu (0.9832) - reversing this operation.

added an activation after the second convolution (0.9847)

changed the first maxpooling to averagepooling (0.9815) - reverting back to MaxPool

changed the second maxpooling to averagepooling (0.9551) - reverting back to MaxPool

changed the one to last activation to sigmoid (0.9801) - reverting back to relu

changed the size of the fully connected layer to 256 (0.9816) - reverting back

adding another fully connected layer with 64 nodes + relu activation (0.9786) - reverting back

adding another fully connected layer with 64 nodes + sigmoid activation (0.9782) - reverting back

changing the last activation to sigmoid (0.9825)

increased number of epochs to 20 (0.9935)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [29]:
# TODO: Load test data

with open(work_dir+'test.p', 'rb') as f:
    data = pickle.load(f)
    
X_test = data['features']
y_test = data['labels']
print(X_all.shape, y_all.shape)
    
# TODO: Preprocess data & one-hot encode the labels

X_test_norm = X_test - 127.5
X_test_norm /= 255

enc = OneHotEncoder(dtype=np.float32)
y_test_ohe = enc.fit_transform(y_test.reshape(-1,1)).toarray()

# TODO: Evaluate model on test data

score = model.evaluate(X_test_norm, y_test_ohe, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])


(12630, 32, 32, 3) (12630,)
12630/12630 [==============================] - 8s     
Test score: 0.150257152031
Test accuracy: 0.957719714936


**Test Accuracy:** (0.9577)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.